# This notebook calculates the emergency performance grant metric sourced from FEMA
* Total amount of emergency performance grant funding recieved per CA county between 2014-2022


In [1]:
import pandas as pd
import os
import sys
import math

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import (
    pull_csv_from_directory, upload_csv_aws, filter_counties
)

In [2]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2a_subset/governance/community_preparedness/fema/emergency_management_performance_grants/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'fema_emergency_management_grants_subset.csv'


In [3]:
# read in FEMA emergency performance grants data
emergency_grants_data = pd.read_csv('fema_emergency_management_grants_subset.csv')
print(len(emergency_grants_data))
emergency_grants_data.head(5)
os.remove('fema_emergency_management_grants_subset.csv')

1280


In [4]:
emergency_grants_data.head()

,id,reportingPeriod,state,legalAgencyName,projectType,projectStartDate,projectEndDate,nameOfProgram,fundingAmount
0,1f318b75-85d7-4be0-bc13-a15358d94810,2014 Close-Out,California,EM- Tule River Tribe,Develop/enhance interoperable communications s...,2014-12-01T00:00:00.000Z,2015-03-01T00:00:00.000Z,EMPG,16345.0
1,fcdcdff4-96a5-40ea-87fa-3b583aaa8f2e,2014 Close-Out,California,EM-Agua Caliente Band of Cahuilla Indians,Establish/enhance sustainable homeland securit...,2014-09-01T00:00:00.000Z,2015-08-01T00:00:00.000Z,EMPG,17500.0
2,ce196d31-8065-4413-8c4a-528353812ff6,2014 Close-Out,California,OA-Alameda County,Establish / enhance citizen awareness of emerg...,2014-10-01T00:00:00.000Z,2015-06-01T00:00:00.000Z,EMPG,452545.0
3,3df463ec-5632-440a-bbb2-1f6ac8e9df55,2014 Close-Out,California,OA-Alpine County,Administer and manage the Homeland Security Gr...,2014-07-01T00:00:00.000Z,2015-06-01T00:00:00.000Z,EMPG,23496.0
4,93c01d7e-9ce3-4a31-8b2e-38bc61cc943f,2014 Close-Out,California,OA-Alpine County,Develop / enhance homeland security / emergenc...,2014-07-01T00:00:00.000Z,2015-06-01T00:00:00.000Z,EMPG,21403.0


In [5]:
# Create a new column 'county' by extracting characters after the first '-'
emergency_grants_data['county'] = emergency_grants_data['legalAgencyName'].str.split('-').str[1]

# Remove any mention of 'county' within the legalAgencyName column
emergency_grants_data['county'] = emergency_grants_data['county'].str.replace(' county', '', case=False)

# Drop any leading or trailing whitespace from the 'county' column
emergency_grants_data['county'] = emergency_grants_data['county'].str.strip()

# Display the modified DataFrame
emergency_grants_data.head()


,id,reportingPeriod,state,legalAgencyName,projectType,projectStartDate,projectEndDate,nameOfProgram,fundingAmount,county
0,1f318b75-85d7-4be0-bc13-a15358d94810,2014 Close-Out,California,EM- Tule River Tribe,Develop/enhance interoperable communications s...,2014-12-01T00:00:00.000Z,2015-03-01T00:00:00.000Z,EMPG,16345.0,Tule River Tribe
1,fcdcdff4-96a5-40ea-87fa-3b583aaa8f2e,2014 Close-Out,California,EM-Agua Caliente Band of Cahuilla Indians,Establish/enhance sustainable homeland securit...,2014-09-01T00:00:00.000Z,2015-08-01T00:00:00.000Z,EMPG,17500.0,Agua Caliente Band of Cahuilla Indians
2,ce196d31-8065-4413-8c4a-528353812ff6,2014 Close-Out,California,OA-Alameda County,Establish / enhance citizen awareness of emerg...,2014-10-01T00:00:00.000Z,2015-06-01T00:00:00.000Z,EMPG,452545.0,Alameda
3,3df463ec-5632-440a-bbb2-1f6ac8e9df55,2014 Close-Out,California,OA-Alpine County,Administer and manage the Homeland Security Gr...,2014-07-01T00:00:00.000Z,2015-06-01T00:00:00.000Z,EMPG,23496.0,Alpine
4,93c01d7e-9ce3-4a31-8b2e-38bc61cc943f,2014 Close-Out,California,OA-Alpine County,Develop / enhance homeland security / emergenc...,2014-07-01T00:00:00.000Z,2015-06-01T00:00:00.000Z,EMPG,21403.0,Alpine


In [6]:
# Extract only the year from the 'reportingPeriod' column and place into new 'year' column
emergency_grants_data['year'] = emergency_grants_data['reportingPeriod'].str.extract(r'(\d{4})')
emergency_grants_data['year'] = emergency_grants_data['year'].str.replace('-01-01', '', case=False)

emergency_grants_data.head(5)

,id,reportingPeriod,state,legalAgencyName,projectType,projectStartDate,projectEndDate,nameOfProgram,fundingAmount,county,year
0,1f318b75-85d7-4be0-bc13-a15358d94810,2014 Close-Out,California,EM- Tule River Tribe,Develop/enhance interoperable communications s...,2014-12-01T00:00:00.000Z,2015-03-01T00:00:00.000Z,EMPG,16345.0,Tule River Tribe,2014
1,fcdcdff4-96a5-40ea-87fa-3b583aaa8f2e,2014 Close-Out,California,EM-Agua Caliente Band of Cahuilla Indians,Establish/enhance sustainable homeland securit...,2014-09-01T00:00:00.000Z,2015-08-01T00:00:00.000Z,EMPG,17500.0,Agua Caliente Band of Cahuilla Indians,2014
2,ce196d31-8065-4413-8c4a-528353812ff6,2014 Close-Out,California,OA-Alameda County,Establish / enhance citizen awareness of emerg...,2014-10-01T00:00:00.000Z,2015-06-01T00:00:00.000Z,EMPG,452545.0,Alameda,2014
3,3df463ec-5632-440a-bbb2-1f6ac8e9df55,2014 Close-Out,California,OA-Alpine County,Administer and manage the Homeland Security Gr...,2014-07-01T00:00:00.000Z,2015-06-01T00:00:00.000Z,EMPG,23496.0,Alpine,2014
4,93c01d7e-9ce3-4a31-8b2e-38bc61cc943f,2014 Close-Out,California,OA-Alpine County,Develop / enhance homeland security / emergenc...,2014-07-01T00:00:00.000Z,2015-06-01T00:00:00.000Z,EMPG,21403.0,Alpine,2014


In [7]:
emergency_grants_data['year'] = emergency_grants_data['reportingPeriod'].str.extract(r'(\d{4})')

# Display unique years
unique_years = emergency_grants_data['year'].unique()
print(unique_years)

['2014' '2016' '2017' '2018' '2019' '2020' '2021' '2022']


In [8]:
# Isolate desired columns for CRI metric
cleaned_emergency_grants = emergency_grants_data[['county', 'year','fundingAmount']]
print(len(cleaned_emergency_grants))
cleaned_emergency_grants.head(10)

1280


,county,year,fundingAmount
0,Tule River Tribe,2014,16345.0
1,Agua Caliente Band of Cahuilla Indians,2014,17500.0
2,Alameda,2014,452545.0
3,Alpine,2014,23496.0
4,Alpine,2014,21403.0
5,Alpine,2014,21873.0
6,Alpine,2014,37652.0
7,Amador,2014,97376.0
8,Butte,2014,918.0
9,Butte,2014,97997.0


In [9]:
# Call filter_counties function to omit of some tribal and non-county entries
cri_emergency_grants = filter_counties(cleaned_emergency_grants, 'county')
print(len(cri_emergency_grants))
cri_emergency_grants.head(5)

1241


,county,year,fundingAmount
2,Alameda,2014,452545.0
3,Alpine,2014,23496.0
4,Alpine,2014,21403.0
5,Alpine,2014,21873.0
6,Alpine,2014,37652.0


In [10]:
# Group the DataFrame by 'county' and 'year', then sum the 'damaged_destroyed_structures' and 'total_fatalities'
cri_emergency_grants = cri_emergency_grants.groupby(['county', 'year']).agg({'fundingAmount': 'sum'}).reset_index()

print(len(cri_emergency_grants))
cri_emergency_grants.head(10)

452


,county,year,fundingAmount
0,Alameda,2014,452545.0
1,Alameda,2016,455793.0
2,Alameda,2017,459407.0
3,Alameda,2018,460021.0
4,Alameda,2019,460021.0
5,Alameda,2020,460874.0
6,Alameda,2021,534974.0
7,Alameda,2022,551584.0
8,Alpine,2014,104424.0
9,Alpine,2016,125233.0


In [11]:
# Group the data further by just county and sum the funding recieved for each county from 2014-2022 (minus 2015)
cri_emergency_grants_total_county = cri_emergency_grants.groupby(['county']).agg({'fundingAmount': 'sum'}).reset_index()
cri_emergency_grants_total_county.rename(columns={'fundingAmount': 'funding_2014_2022'}, inplace=True)

print(len(cri_emergency_grants_total_county))
cri_emergency_grants_total_county.head()

57


,county,funding_2014_2022
0,Alameda,3835219.0
1,Alpine,1041123.0
2,Amador,968348.0
3,Butte,1361817.0
4,Calaveras,1110376.0


In [12]:
# Saving metric df to .csv file
cri_emergency_grants_total_county.to_csv('governance_emergency_management_metric.csv')

In [13]:
# upload final csv file to aws
bucket_name = 'ca-climate-index'
file_name = ['governance_emergency_management_metric.csv']
directory = '3_fair_data/index_data'

#@append_metadata
upload_csv_aws(file_name, bucket_name, directory)
os.remove('governance_emergency_management_metric.csv')

governance_emergency_management_metric.csv uploaded to AWS
